In [2]:
%load_ext lab_black

In [3]:
import re
import json

from uuid import uuid4

from django.db import connections

from rich import print

# Manual Stuff

* Set DELETE_WAGTAIL_IMAGES=False to deactivate post_delete_file_cleanup, to avoid deleting original 🥶
* Start only postgres and then: dropdb python_podcast && createdb python_podcast && python manage.py migrate
* Sometimes after convert: python manage.py fixtree
* Sometimes, you have to run: python manage.py sqlsequencereset cast + execute via pgcli

# Create New Empty DB

In [4]:
import os

from django.conf import settings
from django.core.management import call_command

from pathlib import Path

In [5]:
current_working_dir = Path.cwd()
os.chdir(settings.ROOT_DIR)
!dropdb python_podcast && createdb python_podcast
call_command("migrate")
os.chdir(current_working_dir)

Operations to perform:
  Apply all migrations: account, admin, auth, authtoken, cast, contenttypes, django_comments, filepond, fluent_comments, indieweb, sessions, sites, socialaccount, taggit, threadedcomments, users, wagtailadmin, wagtailcore, wagtaildocs, wagtailembeds, wagtailforms, wagtailimages, wagtailredirects, wagtailsearch, wagtailusers, watson
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0001_initial... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying users.0001_initial... OK
  Applying account.0001_initial... OK
  Applying accou

# Fetch Legacy Data from Database and Restore

In [19]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [dict(zip(columns, row)) for row in cursor.fetchall()]


class Legacy:
    def __init__(self, db_name="legacy"):
        self.db_name = db_name
        self.users = self.fetch_rows("select * from users_user")
        self.blogs = self.fetch_rows("select * from cast_blog")
        self.posts = self.fetch_rows("select * from cast_post")
        self.images = self.fetch_rows("select * from cast_image")
        self.galleries = self.fetch_rows("select * from cast_gallery")
        self.gallery_images = self.fetch_rows("select * from cast_gallery_images")
        self.videos = self.fetch_rows("select * from cast_video")
        self.audios = self.fetch_rows("select * from cast_audio")
        self.chaptermarks = self.fetch_rows("select * from cast_chaptermark")
        self.itunes_artworks = self.fetch_rows("select * from cast_itunesartwork")
        self.comments = self.fetch_rows("select * from django_comments")
        self.comment_flags = self.fetch_rows("select * from django_comment_flags")
        self.threadedcomments_comments = self.fetch_rows(
            "select * from threadedcomments_comment"
        )

    def fetch_rows(self, stmt):
        with connections[self.db_name].cursor() as cursor:
            cursor.execute(stmt)
            rows = dictfetchall(cursor)
        return rows


class Converter:
    def __init__(self, legacy):
        self.legacy = legacy
        self.blog_content_type = ContentType.objects.get(app_label="cast", model="blog")

    def users(self):
        for l_user in self.legacy.users:
            user = User(**l_user)
            user.save()
        return {user.pk: user for user in User.objects.all()}

    def itunes_artworks(self):
        for l_iaw in self.legacy.itunes_artworks:
            iaw = ItunesArtWork(**l_iaw)
            iaw.save()

    def blogs(self):
        blog_legacy_to_wagtail = {}
        root = Page.objects.get(title="Welcome to your new Wagtail site!")
        for l_blog in self.legacy.blogs:
            kwargs = l_blog.copy()
            del kwargs["user_id"]
            del kwargs["id"]
            kwargs["owner"] = self.user_lookup[l_blog["user_id"]]
            kwargs["last_published_at"] = l_blog["modified"]
            # l_blog["content_type"] = self.blog_content_type
            blog = Blog(**kwargs)
            blog = root.add_child(instance=blog)
            blog_legacy_to_wagtail[l_blog["id"]] = blog.pk
        return blog_legacy_to_wagtail

    def images(self):
        for num, l_image in enumerate(self.legacy.images):
            image = Image(
                pk=l_image["id"],
                file=l_image["original"],
                uploaded_by_user=self.user_lookup[l_image["user_id"]],
                created_at=l_image["created"],
                width=l_image["original_width"],
                height=l_image["original_height"],
            )
            image.save()
            if num % 300 == 0:
                print(num)

    def galleries(self):
        for l_gallery in self.legacy.galleries:
            kwargs = {k: v for k, v in l_gallery.items() if k != "user_id"}
            gallery = Gallery(**kwargs)
            gallery.save()

    def gallery_image_links(self):
        links = [
            (gi["id"], gi["gallery_id"], gi["image_id"])
            for gi in self.legacy.gallery_images
        ]
        stmt = "insert into cast_gallery_images (id, gallery_id, image_id) values (%s, %s, %s)"
        with connections["default"].cursor() as cursor:
            cursor.executemany(stmt, links)

    def audios(self):
        for l_audio in self.legacy.audios:
            audio = Audio(**l_audio)
            audio.save()

    def chaptermarks(self):
        for l_mark in self.legacy.chaptermarks:
            mark = ChapterMark(**l_mark)
            mark.save()

    def videos(self):
        for num, video in enumerate(self.legacy.videos):
            upload_user = self.user_lookup[video["user_id"]]
            video = Video(
                pk=video["id"],
                title=Path(video["original"]).name,
                user=upload_user,
                poster=video["poster"],
                poster_seconds=video["poster_seconds"],
                original=video["original"],
                created=video["created"],
                modified=video["modified"],
            )
            video.save(poster=False)
            if num % 50 == 0:
                print(num)

    def convert(self):
        self.user_lookup = self.users()
        self.itunes_artworks()
        self.blog_lookup = self.blogs()
        self.images()
        self.galleries()
        self.gallery_image_links()
        self.videos()
        self.audios()
        self.chaptermarks()

In [20]:
converter = Converter(Legacy())

In [8]:
converter.legacy.itunes_artworks

[{'id': 1,
  'created': datetime.datetime(2018, 11, 21, 9, 18, 45, 339582, tzinfo=datetime.timezone.utc),
  'modified': datetime.datetime(2018, 11, 30, 14, 40, 34, 546525, tzinfo=datetime.timezone.utc),
  'original': 'cast_images/itunes_artwork/pp_itunes_artwork_3k.png',
  'original_height': 3000,
  'original_width': 3000}]

In [9]:
%%time
converter.convert()

0

0

CPU times: user 2.12 s, sys: 566 ms, total: 2.69 s
Wall time: 879 ms


In [17]:
def convert_audios(audios):
    for l_audio in audios:
        audio = Audio(**l_audio)
        audio.save()

In [18]:
convert_audios([converter.legacy.audios[0]])

In [10]:
# converter.legacy.audios

# Migrate Posts

In [11]:
def is_tag(text):
    return text.startswith("{%") and text.endswith("%}")


def tag_to_block(tag):
    tag_name, tag_id = tag.strip("{%").strip("%}").split()
    tag_id = int(tag_id)
    if tag_name == "gallery":
        image_blocks = []
        for image in Gallery.objects.get(pk=tag_id).images.all():
            image_blocks.append({"type": "item", "value": image.pk, "id": str(uuid4())})
        return {"type": tag_name, "value": image_blocks}
    return {"type": tag_name, "value": tag_id}


def content_to_blocks(content):
    blocks = []
    just_tag = re.compile(r"({% \w+ \d+ %})")
    for part in just_tag.split(content):
        if len(part) == 0:
            continue
        if is_tag(part):
            blocks.append(tag_to_block(part))
        else:
            blocks.append({"type": "paragraph", "value": part})
    return blocks


def content_to_streamfield(content):
    detail_blocks = None
    pattern = r"(?P<overview>.*){% if include_detail %}(?P<detail>.*){% endif %}.*"
    match = re.search(pattern, content, re.DOTALL)
    if match is not None:
        overview_blocks = content_to_blocks(match.group("overview"))
        detail_blocks = content_to_blocks(match.group("detail"))
    else:
        overview_blocks = content_to_blocks(content)
    streamfield = [{"type": "overview", "value": overview_blocks}]
    if detail_blocks is not None:
        streamfield.append({"type": "detail", "value": detail_blocks})
    return streamfield


def build_post_from_legacy(legacy, user_lookup):
    # post = Post(title=legacy["title"], visible_date=legacy["visible_date"])
    kwargs = {
        k: v
        for k, v in legacy.items()
        if k not in ["content", "author_id", "blog_id", "id"]
    }
    kwargs["content_type"] = ContentType.objects.get(app_label="cast", model="post")
    kwargs["live"] = legacy["pub_date"] is not None
    if kwargs["live"]:
        kwargs["first_published_at"] = legacy["pub_date"]
        kwargs["last_published_at"] = legacy["modified"]
    kwargs["latest_revision_created_at"] = legacy["modified"]
    kwargs["owner"] = user_lookup[legacy["author_id"]]
    # print(kwargs)
    post = Post(**kwargs)
    raw_body = content_to_streamfield(legacy["content"])
    post.body = json.dumps(raw_body)
    return post

In [12]:
legacy_post_lookup = {p["id"]: p for p in converter.legacy.posts}

In [13]:
# for lpost in legacy_post_lookup.values():
#     if "video" in lpost["content"] and lpost["blog_id"] == 1:
#         print(lpost)

In [14]:
# for lpost in legacy_post_lookup.values():
#     if "include_detail" in lpost["content"]:
#         print(lpost)

In [15]:
%%time
blog_lookup = {k: Page.objects.get(pk=v) for k, v in converter.blog_lookup.items()}
legacy_post_lookup = {p["id"]: p for p in converter.legacy.posts}
# for p_id in (9,):
# for p_id in (183,):
# for p_id in (9, 183, 332, 333):
# for p_id in (329,):
for num, p_id in enumerate(legacy_post_lookup.keys()):
    l_post = legacy_post_lookup[p_id]
    blog = blog_lookup[l_post["blog_id"]]
    post = build_post_from_legacy(l_post, converter.user_lookup)
    post = blog.add_child(instance=post)
    if num % 20 == 0:
        print(num)

0

20

40

CPU times: user 387 ms, sys: 16.1 ms, total: 403 ms
Wall time: 522 ms


In [13]:
[l_post] = [p for p in converter.legacy.posts if p["id"] == 329]

In [14]:
l_post

{'id': 329,
 'created': datetime.datetime(2022, 10, 22, 10, 37, 0, 444369, tzinfo=datetime.timezone.utc),
 'modified': datetime.datetime(2022, 10, 22, 10, 56, 8, 61463, tzinfo=datetime.timezone.utc),
 'title': 'Django Beginner Series: Update Python',
 'pub_date': None,
 'visible_date': datetime.datetime(2022, 10, 22, 0, 0, tzinfo=datetime.timezone.utc),
 'content': 'cd ~/.pyenv<br />\r\ngit pull<br />\r\n<br />\r\npyenv install 3.10.8<br />\r\n<br />\r\npyenv global 3.10.8',
 'slug': 'update-python',
 'author_id': 1,
 'blog_id': 1,
 'podcast_audio_id': None,
 'uuid': UUID('bf0666a1-62d2-438d-9d11-1d195388b68d'),
 'keywords': '',
 'explicit': 1,
 'block': False,
 'comments_enabled': True}

In [29]:
counter = {}
for page in Page.objects.all():
    counter[page.owner] = counter.get(page.owner, 0) + 1
print(counter)

{None: 321, <User: jochen>: 2}

In [36]:
Post.objects.first().visible_date

datetime.datetime(2018, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)

In [15]:
Page.objects.all()[5].post.visible_date

datetime.datetime(2018, 1, 24, 0, 0, tzinfo=datetime.timezone.utc)

In [13]:
# for page in Page.objects.all():
#    print(page.first_published_at)

# Migrate Comments

In [ ]:
ThreadedComment

In [101]:
from django_comments.models import Comment


def convert_comments(comments):
    for l_comment in comments:
        comment = Comment(**l_comment)
        comment.content_type_id = 66
        comment.save()


def convert_comment_flags(comment_flags):
    for l_comment_flag in comment_flags:
        flag = CommentFlag(**l_comment_flag)
        flag.save()


def convert_threaded_comments(threaded_comments):
    # does not work due to db constraints
    for l_threaded_comment in threaded_comments:
        threaded_comment = ThreadedComment(**l_threaded_comment)
        threaded_comment.save()

In [36]:
convert_comments(converter.legacy.comments)

In [37]:
convert_comment_flags(converter.legacy.comment_flags)

In [41]:
Comment.objects.first().content_type

<ContentType: authtoken | token>

Copy threaded_comments manually:
```
pg_dump -a -t threadedcomments_comment python_podcast_legacy | psql python_podcast
```

# Backup migrated Database

Before applying the backup, change the site name in wagtail settings to the new fqdn + port 443. Otherwise item.get_full_url() will not be correct.

```
pg_dump python_podcast | gzip > backups/db.staging.psql.gz
cd deploy
ansible-playbook restore_database.yml
```